# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
import kaggle
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.automl import AutoMLConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from trainCovid19Dataset import clean_data
import os
import shutil

ModuleNotFoundError: No module named 'azureml.train.automl'

## Initialize Workspace

In [ ]:
# Create a workspace, if it doesn't exist, using the AzureML SDK
if (workspace.get(name = 'CapstoneWorkspace') == False ):
    ws = Workspace.create(name='CapstoneWorkspace',
               subscription_id='<azure-subscription-id>',
               resource_group='myresourcegroup',
               create_resource_group=True,
               location='eastus2'
               )
else:
    ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

project_folder = './capstone-project'
# choose a name for experiment
experiment_name = 'Covid19VaccinationExperiment'
experiment=Experiment(ws, experiment_name)
experiment

## Create Cluster

Get cluster if it exists else create one

In [ ]:
# Create compute cluster
cpu_cluster_name = "Covid19Cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4,identity_type="SystemAssigned")
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)
# Get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

## Dataset

### Overview

I Chose a COVID-19 World Vaccination Dataset that holds a track of the world vaccination including the name of the country, Which vaccines have been used, the use of the vaccine by city, and where the vaccine is more used.

Since the covid-19 vaccination is among the hottest subjects in the world, and as a member of the society being interested in such statistic calculations can help further scientists or even regular people to better understand the global effect of this vaccine all over the world.

I used Kaggle's API to download the Dataset.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19Dataset"
description_text = "Covid19 Vaccination DataSet from Kaggle"

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    os.environ['KAGGLE_USERNAME']= 'ocherif'
    os.environ['KAGGLE_KEY']= 'cb037f99cae382b7a67c68f8048e01be'
    import kaggle
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('gpreda/covid-world-vaccination-progress', path='kaggle/', unzip=True)
    
    ds = pd.read_csv("./starter_file/kaggle/country_vaccinations.csv")  
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe().dropna() 
df.describe()

In [ ]:
# preview the first 10 rows of the dataset
df.head(10)

In [ ]:
# Use the clean_data function to clean your data.
x, y = clean_data(df)
data = pd.concat([x,y],axis=1)
data.head()

In [ ]:
# Split data into train and test sets.
training_data,validation_data = train_test_split(data,test_size = 0.3,random_state = 42,shuffle=True)

In [ ]:
# Create necessary folders
if "training" not in os.listdir():
    os.mkdir("./training")
if "data" not in os.listdir():
    os.mkdir("./data")
# store training_dataset into it using datastore
script_folder = './training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('trainCovid19Dataset.py', script_folder)

## AutoML Configuration
TODO: Explain why you chose the automl settings and cofiguration you used below.
The settings used below refers to a classification task within a number of settings chosen based on the existing workspace and cluster configuration restrictions 

In [ ]:
#convert the training dataset to a CSV file and store it under the training folder
training_data.to_csv('training/training_data.csv')
#Create an experiment for the AutoML testing script
exp = Experiment(workspace=ws, name="Covid19AutoMlExperiment")
datastore = ws.get_default_datastore()
datastore.upload(src_dir='training/',target_path='data/')
# Get the dataset from the data folder
training_dataset = TabularDatasetFactory.from_delimited_files(path=[(datastore,('data/training_data.csv'))])
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "experiment_timeout_hours": 0.5,
    "max_concurrent_iterations": 8,
    "max_cores_per_iteration": -1,
    "verbosity": logging.INFO,
}
automl_config = AutoMLConfig(task = 'classification',
                             compute_target = compute_target,
                             training_data = training_dataset,
                             label_column_name = 'used_Vaccine',
                             featurization= 'auto',
                             iterations=40,
                             path=project_folder,
                             debug_log = "Covid_automl_errors.log",
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
tag = {"Covid19": "Capstone project: Covid19 AutoML Experiment"}
remote_run = experiment.submit(automl_config,tags=tag, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

In [ ]:
# Retrieve and save best model.
best_automl_run, model = remote_run.get_output()

# Get the metrics of the best selected run
best_run_metrics = best_automl_run.get_metrics()
best_automl_run.final_estimator

# Show the Accuracy of that run
print('Best accuracy: {}'.format(best_run_metrics['Accuracy']))

## Model Deployment

Remember you have to deploy only one of the two models you trained. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# register the best model
best_automl_run.register_model(model_name = "automl_covid19_model", model_path = './outputs/')

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Covid19Cluster"

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service